In [81]:
import os
from PIL import Image

# 디렉토리 경로 설정
directory = 'C:/Users/USER/Desktop/트레인 데이터/전공드라이버 - 복사본'

# 디렉토리 내 파일 목록 가져오기
files = os.listdir(directory)

# 파일 변환 함수
def convert_to_jpg(file_path, output_path):
    try:
        with Image.open(file_path) as img:
            # PNG와 JPG를 JPG로 변환
            img.convert('RGB').save(output_path, 'JPEG')
        print(f"Converted {file_path} to {output_path}")
        return True
    except Exception as e:
        print(f"Error converting {file_path}: {e}")
        return False

# 파일 변환
for filename in files:
    file_path = os.path.join(directory, filename)
    file_extension = os.path.splitext(filename)[1].lower()
    
    # PNG와 JPG 파일만 변환
    if file_extension in ['.png', '.jpg', '.jpeg']:
        new_filename = os.path.splitext(filename)[0] + '.jpg'
        new_file_path = os.path.join(directory, new_filename)
        
        if convert_to_jpg(file_path, new_file_path):
            # 변환 성공 시, 원래의 PNG 파일 삭제
            if file_extension == '.png':
                os.remove(file_path)
                print(f"Deleted original file: {file_path}")

print("모든 파일이 JPG 형식으로 변환되었으며, PNG 파일은 삭제되었습니다.")


Converted C:/Users/USER/Desktop/트레인 데이터/전공드라이버 - 복사본\1.jpg to C:/Users/USER/Desktop/트레인 데이터/전공드라이버 - 복사본\1.jpg
Converted C:/Users/USER/Desktop/트레인 데이터/전공드라이버 - 복사본\10.jpg to C:/Users/USER/Desktop/트레인 데이터/전공드라이버 - 복사본\10.jpg
Converted C:/Users/USER/Desktop/트레인 데이터/전공드라이버 - 복사본\11.jpg to C:/Users/USER/Desktop/트레인 데이터/전공드라이버 - 복사본\11.jpg
Converted C:/Users/USER/Desktop/트레인 데이터/전공드라이버 - 복사본\12.jpg to C:/Users/USER/Desktop/트레인 데이터/전공드라이버 - 복사본\12.jpg
Converted C:/Users/USER/Desktop/트레인 데이터/전공드라이버 - 복사본\13.jpg to C:/Users/USER/Desktop/트레인 데이터/전공드라이버 - 복사본\13.jpg
Converted C:/Users/USER/Desktop/트레인 데이터/전공드라이버 - 복사본\14.jpg to C:/Users/USER/Desktop/트레인 데이터/전공드라이버 - 복사본\14.jpg
Converted C:/Users/USER/Desktop/트레인 데이터/전공드라이버 - 복사본\15.jpg to C:/Users/USER/Desktop/트레인 데이터/전공드라이버 - 복사본\15.jpg
Converted C:/Users/USER/Desktop/트레인 데이터/전공드라이버 - 복사본\16.jpg to C:/Users/USER/Desktop/트레인 데이터/전공드라이버 - 복사본\16.jpg
Converted C:/Users/USER/Desktop/트레인 데이터/전공드라이버 - 복사본\17.jpg to C:/Users/USER/Desktop/트레인 데이터/전공드라이

In [82]:
import os
import re

# 디렉토리 경로 설정
directory = "C:/Users/USER/Desktop/트레인 데이터/전공드라이버 - 복사본"

# 디렉토리 내 파일 목록 가져오기
files = os.listdir(directory)

# 정규 표현식으로 파일 이름 정렬 (숫자가 포함된 파일 이름을 기준으로 정렬)
files.sort(key=lambda x: int(re.search(r'\d+', x).group()))

# 파일 이름을 snap_{i+1} 형식으로 변경
for i, filename in enumerate(files):
    # 파일 확장자 추출
    file_extension = os.path.splitext(filename)[1]
    
    # 새 파일 이름 생성
    new_name = f'ball_{i+1}{file_extension}'
    
    # 경로 설정
    old_path = os.path.join(directory, filename)
    new_path = os.path.join(directory, new_name)
    
    # 파일 이름 변경
    os.rename(old_path, new_path)

print("파일 이름 변경 완료")


파일 이름 변경 완료


In [83]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid

In [84]:
ENDPOINT = 'https://b05mlcv20240717.cognitiveservices.azure.com/'
training_key = '2dc402bd0c5e45fbbc54730fb2757532'
prediction_key = '7fd189c12f57491ca0825b87124b2bcb'
prediction_resource_id = '/subscriptions/b850d62a-25fe-4d3a-9697-ea40449528a9/resourceGroups/b05-AzureMLDTest/providers/Microsoft.CognitiveServices/accounts/b05mlcv20240717-Prediction'

In [85]:
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

In [86]:
publish_iteration_name = "b05_od"

# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Create a new project
print ("Creating project...")
# Use uuid to avoid project name collisions.
project = trainer.create_project(str(uuid.uuid4()), domain_id=obj_detection_domain.id)

Creating project...


In [96]:
# Make tags in the new project
ball_tag = trainer.create_tag(project.id, "ball")

In [97]:
ball_image_regions = {f"ball_{i}": [0, 0, 1, 1] for i in range(1, 51)}

In [98]:
base_image_location = "C:/Users/USER/Desktop/트레인 데이터/전공드라이버 - 복사본"
# Go through the data table above and create the images
print ("Adding images...")
tagged_images_with_regions = []#태깅을위한 리스트 초기화

Adding images...


In [99]:
for file_name in ball_image_regions.keys():
    # 'ball_50'까지만 처리
    if int(file_name.split('_')[1]) <= 50:
        x, y, w, h = ball_image_regions[file_name]
        regions = [Region(tag_id=ball_tag.id, left=x, top=y, width=w, height=h)]

# 이미지 파일 경로를 구성
    file_path = os.path.join(base_image_location, file_name + ".jpg")
    
    try:
        with open(file_path, mode="rb") as image_contents:
            # 이미지와 region을 함께 저장
            tagged_images_with_regions.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), regions=regions))
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {file_path}")
    except Exception as e:
        print(f"파일 처리 중 오류 발생: {e}")


print("이미지 파일 처리가 완료되었습니다.")

이미지 파일 처리가 완료되었습니다.


In [100]:
upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=tagged_images_with_regions))
if not upload_result.is_batch_successful:
    print("Image batch upload failed.")
    for image in upload_result.images:
        print("Image status: ", image.status)
    exit(-1)

Image batch upload failed.
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OKDuplicate
Image status:  OK
Image status:  OK
Image status:  OKDuplicate
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
Image status:  OK
